In [21]:
# ===========================
# 03_Metrics_Comparison.ipynb
# ===========================

import os, glob
import numpy as np
import pandas as pd
import cv2
from skimage.color import rgb2lab, deltaE_ciede2000
from skimage.metrics import structural_similarity as ssim

# ---------- Paths ----------
PROJECT_DIR  = "/content/drive/MyDrive/Colab Notebooks/final_project_computer_vision" #Change this to the root of where the poject is
DATA_DIR     = f"{PROJECT_DIR}/data"
IMAGES_DIR   = f"{DATA_DIR}/images"
OUT_DIR      = f"{PROJECT_DIR}/outputs"
RECOLOR_DIR  = f"{OUT_DIR}/recolor"
os.makedirs(RECOLOR_DIR, exist_ok=True)

# ---------- Helpers ----------
def ensure_mask_size(mask_u8: np.ndarray, H: int, W: int) -> np.ndarray:
    if mask_u8.ndim == 3:
        mask_u8 = cv2.cvtColor(mask_u8, cv2.COLOR_BGR2GRAY)
    if mask_u8.dtype != np.uint8:
        mask_u8 = mask_u8.astype(np.uint8)
    if mask_u8.max() in (0, 1):
        mask_u8 = (mask_u8 * 255).astype(np.uint8)
    if mask_u8.shape[:2] != (H, W):
        mask_u8 = cv2.resize(mask_u8, (W, H), cv2.INTER_NEAREST)
    return mask_u8

def find_source_image(stem: str) -> str:
    for ext in (".jpg", ".jpeg", ".png"):
        p = os.path.join(IMAGES_DIR, stem + ext)
        if os.path.isfile(p): return p
    raise FileNotFoundError(f"Original image not found for stem='{stem}' in {IMAGES_DIR}")

def load_mask(path: str, H: int, W: int) -> np.ndarray:
    m = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if m is None:
        raise FileNotFoundError(f"Missing mask: {path}")
    return ensure_mask_size(m, H, W)

def hex_to_rgb(hex_color: str) -> np.ndarray:
    hex_color = hex_color.strip().replace("#","")
    return np.array([int(hex_color[0:2],16), int(hex_color[2:4],16), int(hex_color[4:6],16)], dtype=np.uint8)

def load_recolor(stem: str, variant: str, space: str) -> np.ndarray:
    p = os.path.join(RECOLOR_DIR, f"{stem}_{variant}_{space}.png")
    x = cv2.imread(p)
    if x is None:
        raise FileNotFoundError(f"Missing recolor: {p}")
    return cv2.cvtColor(x, cv2.COLOR_BGR2RGB)

# ---------- Metrics ----------
def compute_color_metrics(img_rgb, rec_rgb, mask_u8, target_rgb, leakage_deltaE_thresh=2.0):
    H, W = img_rgb.shape[:2]
    img01 = img_rgb.astype(np.float32)/255.0
    rec01 = rec_rgb.astype(np.float32)/255.0
    mask  = (mask_u8 > 0).astype(np.uint8)

    ssim_val = ssim(img01, rec01, channel_axis=2, data_range=1.0)

    outside = (mask == 0)
    diff2 = (img01 - rec01)**2
    if outside.sum() < 10:
        outside_psnr = float('inf')
    else:
        outside_mse  = float(diff2[outside].mean())
        outside_psnr = float(20*np.log10(1.0) - 10*np.log10(outside_mse)) if outside_mse > 1e-12 else float('inf')

    lab_rec = rgb2lab(rec01)
    lab_tgt = rgb2lab(np.tile((target_rgb/255.0)[None,None,:], (H,W,1)))
    dE_target = deltaE_ciede2000(lab_rec, lab_tgt)
    inside = (mask == 1)
    dE_inside_to_target_mean = float(np.mean(dE_target[inside])) if inside.sum() >= 10 else float('nan')

    lab_img   = rgb2lab(img01)
    dE_change = deltaE_ciede2000(lab_img, lab_rec)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    ring = cv2.dilate(mask, k, 1).astype(bool) & (~mask.astype(bool))
    leakage_bnd = float('nan') if ring.sum() < 10 else float(100.0*(dE_change[ring] > leakage_deltaE_thresh).mean())

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    ring   = cv2.dilate(mask, kernel, 1).astype(bool) & (~mask.astype(bool))

    BoundaryDeltaE_mean = float(np.mean(dE_change[ring])) if ring.sum()>=10 else float('nan')

    return {
        "SSIM_full": float(ssim_val),
        "PSNR_outside": float(outside_psnr),
        "DeltaE_inside_to_target_mean": float(dE_inside_to_target_mean),
        "Leakage_pct_boundary": float(leakage_bnd),
        "BoundaryDeltaE_mean": BoundaryDeltaE_mean,
    }


def compute_edge_alignment(img_rgb: np.ndarray, mask_u8: np.ndarray,
                           canny_low: int = 50, canny_high: int = 150, hit_radius: int = 2) -> dict:
    """
    Mask-only quality (alignment with image edges):
      • edge_mean_dist_px / edge_p95_dist_px: distance from mask boundary to nearest image edge (↓ better)
      • edge_hit_rate: fraction of boundary pixels within hit_radius of any edge (↑ better)
    """
    H, W = img_rgb.shape[:2]
    mask = (ensure_mask_size(mask_u8, H, W) > 0).astype(np.uint8)

    gray  = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, canny_low, canny_high)
    inv   = (edges == 0).astype(np.uint8)
    dist  = cv2.distanceTransform(inv, cv2.DIST_L2, 3)

    k3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    boundary = (cv2.dilate(mask, k3, 1) > 0) & (cv2.erode(mask, k3, 1) == 0)

    if boundary.sum() < 5:
        return {"edge_mean_dist_px": float('nan'), "edge_p95_dist_px": float('nan'), "edge_hit_rate": float('nan')}

    dvals = dist[boundary]
    mean_d = float(np.mean(dvals))
    p95_d  = float(np.percentile(dvals, 95))

    if hit_radius <= 0:
        hit_rate = float(np.mean(edges[boundary] > 0))
    else:
        kr = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*hit_radius+1, 2*hit_radius+1))
        edges_dil = cv2.dilate((edges > 0).astype(np.uint8), kr, 1).astype(bool)
        hit_rate = float(np.mean(edges_dil[boundary]))

    return {"edge_mean_dist_px": mean_d, "edge_p95_dist_px": p95_d, "edge_hit_rate": hit_rate}

# ---------- Parameters ----------
target_hex = "#2E86DE"
target_rgb = hex_to_rgb(target_hex)
leakage_deltaE_thresh = 2.0

VARIANTS     = ["BodyRaw", "BodyGC"]
COLOR_SPACES = ["HSV", "Lab"]

# ---------- Collect per-image metrics ----------
color_rows = []
seg_rows   = []

# stems are per-image subfolders created by 01_Segmentation
stems = [d for d in os.listdir(OUT_DIR) if os.path.isdir(os.path.join(OUT_DIR, d)) and d.lower() != "recolor"]
stems.sort()

for stem in stems:
    stem_dir = os.path.join(OUT_DIR, stem)

    try:
        img_path = find_source_image(stem)
    except FileNotFoundError as e:
        print(f"Skipping '{stem}': {e}")
        continue

    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"Skipping '{stem}': cannot read image.")
        continue
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    H, W = img_rgb.shape[:2]

    try:
        m_raw   = load_mask(os.path.join(stem_dir, "body_raw.png"),   H, W)
        m_gc    = load_mask(os.path.join(stem_dir, "body_gc.png"),    H, W)
    except FileNotFoundError as e:
        print(f"Skipping '{stem}': {e}")
        continue

    masks_by_variant = {"BodyRaw": m_raw, "BodyGC": m_gc}

    # COLOR METRICS (for each recolor space and variant)
    for variant in VARIANTS:
        for space in COLOR_SPACES:
            try:
                rec = load_recolor(stem, variant, space)
            except FileNotFoundError as e:
                continue
            m = compute_color_metrics(img_rgb, rec, masks_by_variant[variant], target_rgb, leakage_deltaE_thresh)
            m.update({"image": stem, "mask": variant, "color_space": space})
            color_rows.append(m)

    for variant, msk in masks_by_variant.items():
        met = compute_edge_alignment(img_rgb, msk, canny_low=50, canny_high=150, hit_radius=2)
        seg_rows.append({"image": stem, "mask": variant, **met})

df_color = pd.DataFrame(color_rows)
df_seg   = pd.DataFrame(seg_rows)

color_csv = os.path.join(OUT_DIR, "metrics_color_all.csv")
seg_csv   = os.path.join(OUT_DIR, "metrics_segmentation_all.csv")
df_color.to_csv(color_csv, index=False)
df_seg.to_csv(seg_csv, index=False)

# ---------- Aggregates & Deltas vs BodyRaw ----------
metrics_color = ["SSIM_full","PSNR_outside",
                 "DeltaE_inside_to_target_mean",
                 "Leakage_pct_boundary"]
higher_better = {"SSIM_full", "PSNR_outside"}

delta_rows = []
improve_rows = []

for space in COLOR_SPACES:
    sub = df_color[df_color["color_space"] == space]
    for image in sub["image"].unique():
        sub_img = sub[sub["image"] == image].set_index("mask")

        if not {"BodyRaw", "BodyGC"}.issubset(sub_img.index):
            continue

        raw_vals = sub_img.loc["BodyRaw", metrics_color]
        gc_vals  = sub_img.loc["BodyGC",  metrics_color]

        deltas   = {}
        improves = {}
        for k in metrics_color:
            d = float(gc_vals[k] - raw_vals[k]) if k in higher_better else float(raw_vals[k] - gc_vals[k])
            deltas[f"Δ_{k}"]    = d
            improves[f"improve_{k}"] = (d > 0.0)

        delta_rows.append({"image": image, "color_space": space, **deltas})
        improve_rows.append({"image": image, "color_space": space, **improves})

df_delta_color = pd.DataFrame(delta_rows)
df_improve     = pd.DataFrame(improve_rows)

df_delta_mean = (df_delta_color.groupby(["color_space"], as_index=False)
                 .mean(numeric_only=True))

improve_cols = [c for c in df_improve.columns if c.startswith("improve_")]
df_improve_rate = (df_improve.groupby(["color_space"], as_index=False)[improve_cols]
                   .mean(numeric_only=True))
for c in improve_cols:
    df_improve_rate[c] = (100.0 * df_improve_rate[c]).round(2)

# ---------- Segmentation deltas vs BodyRaw ----------
df_seg_p = df_seg.pivot_table(index=["image"], columns="mask",
                              values=["edge_mean_dist_px","edge_p95_dist_px","edge_hit_rate"])
seg_delta_rows = []
for image in df_seg["image"].unique():
    try:
        p95_raw   = df_seg_p.loc[image, ("edge_p95_dist_px","BodyRaw")]
        p95_gc    = df_seg_p.loc[image, ("edge_p95_dist_px","BodyGC")]
        hit_raw   = df_seg_p.loc[image, ("edge_hit_rate","BodyRaw")]
        hit_gc    = df_seg_p.loc[image, ("edge_hit_rate","BodyGC")]
    except KeyError:
        continue
    seg_delta_rows.append({
        "image": image,
        "Δ_edge_p95_dist_px":  float(p95_raw  - p95_gc),
        "Δ_edge_hit_rate":     float(hit_gc   - hit_raw),
    })
df_delta_seg = pd.DataFrame(seg_delta_rows)

df_delta_seg_mean = df_delta_seg.mean(numeric_only=True).to_frame().T

# ---------- Console summary ----------
def fmt_pct(x):
    return f"{x:.1f}%"

print("== Color improvement rates: BodyGC vs BodyRaw (wins %) ==")
if not df_improve_rate.empty:
    for space in COLOR_SPACES:
        row = df_improve_rate[df_improve_rate["color_space"] == space]
        if row.empty: continue
        row = row.iloc[0]
        print(f"[{space}] "
              f"Leakage_boundary: {fmt_pct(row['improve_Leakage_pct_boundary'])} | "
              f"ΔE_inside_to_target_mean: {fmt_pct(row['improve_DeltaE_inside_to_target_mean'])} | "
              f"SSIM: {fmt_pct(row['improve_SSIM_full'])} | "
              f"PSNR_outside: {fmt_pct(row['improve_PSNR_outside'])} "
              )
else:
    print("(no recolor metrics found)")

print("\n== Mean color deltas (BodyGC − BodyRaw) — positive = better ==")
if not df_delta_mean.empty:
    print(df_delta_mean.to_string(index=False))
else:
    print("(no deltas)")

print("\n== Segmentation deltas (BodyGC − BodyRaw) — positive = better ==")
if not df_delta_seg_mean.empty:
    print(df_delta_seg_mean.to_string(index=False))
else:
    print("(no seg deltas)")

print("\nSaved CSVs in:", OUT_DIR)


== Color improvement rates: BodyGC vs BodyRaw (wins %) ==
[HSV] Leakage_boundary: 0.0% | ΔE_inside_to_target_mean: 63.2% | SSIM: 42.1% | PSNR_outside: 57.9% 
[Lab] Leakage_boundary: 0.0% | ΔE_inside_to_target_mean: 15.8% | SSIM: 36.8% | PSNR_outside: 68.4% 

== Mean color deltas (BodyGC − BodyRaw) — positive = better ==
color_space  Δ_SSIM_full  Δ_PSNR_outside  Δ_DeltaE_inside_to_target_mean  Δ_Leakage_pct_boundary
        HSV     0.000117        0.000794                        0.001765                     0.0
        Lab     0.000211        0.000576                       -0.003927                     0.0

== Segmentation deltas (BodyGC − BodyRaw) — positive = better ==
 Δ_edge_p95_dist_px  Δ_edge_hit_rate
           0.020715        -0.000534

Saved CSVs in: /content/drive/MyDrive/Colab Notebooks/final_project_computer_vision/outputs
